## Regression Loss

### 1. MSE

In [81]:
import numpy as np

y_true_np = np.array([3.0, 5.0, 2.5])
y_pred_np = np.array([2.5, 5.5, 2.0])

def mse_loss(pred, target):
    return np.mean((pred-target)**2)
mse = mse_loss(y_pred_np, y_true_np)
print("MSE:", mse)

MSE: 0.25


In [82]:
import torch

y_true_t = torch.tensor([3.0, 5.0, 2.5])
y_pred_t = torch.tensor([2.5, 5.5, 2.0], requires_grad=True)
# requires_grad: 这个tensor参与梯度计算，需要在反向传播的时候计算他的梯度

def mse_loss(pred, target):
    return torch.mean((pred-target)**2)
mse = mse_loss(y_pred_t, y_true_t)
print("MSE:", mse)

MSE: tensor(0.2500, grad_fn=<MeanBackward0>)


### 2. MAE 

In [83]:
def mae_loss(pred, target):
    return np.mean(np.abs(pred-target))

mae = mae_loss(y_pred_np, y_true_np)
print("MAE:", mae)

MAE: 0.5


In [84]:
def mae_loss(pred, target):
    return torch.mean(torch.abs(pred-target))

mae = mae_loss(y_pred_t, y_true_t)
print("MAE:", mae)

MAE: tensor(0.5000, grad_fn=<MeanBackward0>)


### 3. Huber Loss

In [85]:
def huber_loss(pred, target, delta=1.0):
    diff = np.abs(pred-target)
    loss = np.where(
        diff<=delta, 
        0.5*diff**2, 
        delta*(diff-0.5*delta)
    )
    return np.mean(loss)
huber_loss = huber_loss(y_pred_np, y_true_np)
print("huber loss:", huber_loss)

huber loss: 0.125


In [86]:
def huber_loss(pred, target, delta=1.0):
    diff = torch.abs(pred-target)
    loss = torch.where(
        diff<=delta, 
        0.5*diff**2, 
        delta*(diff-0.5*delta)
    )
    return loss.mean()
huber_loss = huber_loss(y_pred_t, y_true_t)
print("huber loss:", huber_loss)

huber loss: tensor(0.1250, grad_fn=<MeanBackward0>)


### 4. RMSE

In [87]:
def rmse_loss(pred, target):
    return np.sqrt(np.mean((pred-target)**2))
rmse = rmse_loss(y_pred_np, y_true_np)
print("RMSE:", rmse)

RMSE: 0.5


In [88]:
def rmse_loss(pred, target):
    return torch.sqrt(torch.mean((pred-target)**2))
rmse = rmse_loss(y_pred_t, y_true_t)
print("RMSE:", rmse)

RMSE: tensor(0.5000, grad_fn=<SqrtBackward0>)


## Classification Loss

### 1. Binary Cross Entropy

In [89]:
# labels
y_true_np = np.array([1, 0, 1])
y_true_t = torch.tensor([1., 0., 1.])

# logits
logits_np = np.array([0.2, 0.6, 0.2])
logits_t = torch.tensor([0.2, 0.6, 0.2], requires_grad=True)

In [90]:
def bce_loss(pred, target, eps=1e-4):
    return -np.mean(target*np.log(pred+eps)+(1-target)*np.log(1-pred+eps))

bce_loss = bce_loss(logits_np, y_true_np)
print("BCE Loss:", bce_loss)

BCE Loss: 1.3779722793012823


In [91]:
def bce_loss(pred, target, eps=1e-4):
    return torch.mean(target*torch.log(pred+eps)+(1-target)*torch.log(1-pred+eps))
bce_loss = bce_loss(logits_t, y_true_t)
print("BCE Loss:", bce_loss)

BCE Loss: tensor(-1.3780, grad_fn=<MeanBackward0>)


### 2. Categorical Cross-Entropy (multiclass, but have only one label)

In [92]:
# batch=3, classes=4
logits_np = np.array([
    [2.0, 1.0, 0.1, -1.0],  # batch 1
    [0.5, 2.5, 0.3, 0.1], # batch 2
    [1.2, 0.2, 2.0, 0.5] # batch 3
])

labels_np = np.array([0, 1, 2])

logits_t = torch.tensor([
    [2.0, 1.0, 0.1, -1.0],
    [0.5, 2.5, 0.3, 0.1],
    [1.2, 0.2, 2.0, 0.5]
])

labels_t = torch.tensor([0, 1, 2])

In [93]:
def softmax(logits):
    exp = np.exp(logits)
    return exp/np.sum(exp, axis=1, keepdims=True)

def categorical_cross_entropy(logits, labels, eps=1e-12):
    probs = softmax(logits)
    N = logits.shape[0] 
    loss = -np.log(probs[np.arange(N), labels]+eps)
    return np.mean(loss)

ce = categorical_cross_entropy(logits_np, labels_np)
print("CE Loss:", ce)

CE Loss: 0.44939341449096365


In [94]:
def softmax(logits):
    exp = torch.exp(logits)
    return exp/torch.sum(exp, dim=1, keepdims=True)

def categorical_cross_entropy(logits, labels, eps=1e-12):
    probs = softmax(logits)
    N = logits.shape[0]
    loss = -torch.log(probs[torch.arange(N), labels]+eps)
    return loss.mean()

ce = categorical_cross_entropy(logits_t, labels_t)
print("CE Loss:", ce)

CE Loss: tensor(0.4494)


## Ranking Loss

### 1. Pointwise 

#### 1.1. MSE (same as the regression loss)

#### 1.2. MAE (same as the regression loss)

#### 1.3 Binary Cross Entropy (same as the classifaction loss)

### 2. Pairwise